In [1]:
#import the required Python packages

import pandas as pd
#import datetime   #check this
import numpy as np
import math
from pandas import DataFrame

math.exp = np.exp
math.pow = np.power
math.sqrt = np.sqrt

In [2]:
#df=pd.read_excel(r'Data/NWSAS_input_data.xlsx')


In [35]:
import nexus_tool

ModuleNotFoundError: No module named 'nexus_tool'

# This notebook illustrates the main equations used for surface water pumping

**Source**: *The mathematics of pumping water*

This source is a manual for certain calculations which should be backed up with academic literature

# Step1: Calculate the head

Htot = Hstat + Hd + (Prt - Pres)

where: 

Htot: Total head

Hstat: Static head (It can be assumed in this case as the elevation difference)

Hd: Dynamic Head

Prt: Pressure on the surface of water in the receiving  tank.

Pres: Pressure on the surface of water in the reservoir.


## How to calculate the hydraulic head

Darct Weisbach Equation is used to calculate the hydraulic head

Hd = (K*V2)/(2*g)

Where: 

K: loss coefficient
V: velocity in the pipe in (m/sec)
g: Acceleration due to gravity=9.81 (m/sec2)

K = K_fitting + K_pipe

K_fitting: Depends on fitting items * number of items used. (see table in the source)

In long pipelines these local head losses (K_fitting) are often minor in comparison with energy losses due to friction and may be neglected.

so the main losses are due to friction

K_pipe = (f*l)/D 

f: friction coofcient 
L: lenght of the pipe in m
D: Diameter in m. 

f = 0.25/[log{(K/3.7*D)+(5.74/Re**0.9)}]**2

Where: 
K: Roughness factor (m): is a standard value obtained from standard tables and based upon the material of the pipe. 
Re: Reynolds number Re=(V*D)/v  
v: is the kinematic viscosity (m2/sec)

This equation can be simplified for laminar flow as follows: 
f = 64/Re



# Step2: Calculate the water flow

In this case the water flow will be calculated using WEAP. It will represent the flow in (m3/sec) and will be calculated for every pipeline.  



In [31]:
## INPUTS

#H_static = df['elevation']  #this is a gis layer
H_static =10

#L = df['pipe_lenght']      #Lenght of the pipeline segment in m
L=5

#D = df['diameter']        #this will be assumed for each pipeline segment
D=4


g = 9.81               #gravitational acceleration in m/sec2

Ken_visc=1.004**-6          #Kinematic Viscosity in (m2/s) value taken from https://www.engineersedge.com/physics/water__density_viscosity_specific_weight_13146.htm



Q=30                #assumption for testing

In [32]:
pi=3.14
A = (pi*D**2)/4
V = Q/A

dens = 1000 #density of water = 1000 kg/m3
Re = (V*D)/Ken_visc
pump_eff = 0.6

#f =0.25/[log{(k/(3.7*D))+(5.74/(Re*0.9))}]**2
#k = roughness factor k is a standard value obtained from standard tables and is based opon the material of the pipe, including any internal coatings and the internal condition of the pipeline i.e. good, normal or poor. 
f=64/Re

#K_pipe = (f*L)/D



#K_loss = Kfit_tot + K_pipe  #Loss coefficient 

#H_dynamic = (K_loss*V**2)/(2*g)

#OR

H_dynamic = ((f*L*16*Q**2)/((D**5)*2*g*(pi**2)))


H_total = H_static + H_dynamic

In [33]:
H_total

12.377170494165817

# Step3: Calculate energy demand for surface water pumping

P = (Q * H * g * dens)/(pump_eff)

Where: 
P: is the energy requirement in (watts)
Q: water flow rate in (m3/sec)
H: total head in (m)
dens: is the density of water = 1000 kg/m3



In [34]:
P = (Q * H_total * g * dens)/(pump_eff)

P/1000

6071.002127388333

In [27]:
######### input variables ##############3
pumping_hours_per_day=10 #is this an assumption?? 
deff= 1
aeff= 0.45  #Surface irrigation, low efficiency:0.45 , high: 0.65  // Drip irrigation: 0.9
ky_dict = {'dates':0.5,'vegetable':1.1,'olives':0.8} #ky values for each crop
kc_dict = {'dates': [0.8,0.9,1,0.8], 'vegetable':[0.5,1,1,0.8], 'olives':[0.45,0.55,0.55,0.6]} #kc values for each crop in each growing season

#defines the agricultural mode for each region, i.e. specifying the percentage of land cultivated per crop type 
df['Mode']=[{'dates':0.5,'vegetable':0.5,'olives':0}]*df.shape[0]
df.loc[df['NAME_1']=='Jufrah','Mode'] = [{'dates':0.7,'vegetable':0.3,'olives':0}]
df.loc[df['NAME_1']=='Gharyan','Mode'] = [{'dates':0,'vegetable':0.3,'olives':0.7}]

output_file = 'surf45_waterdemand' #output file name

In [31]:
#Effective rainfall function 

def eff_rainfall(prec,eto):
    return (1.253*((prec**0.824)-2.935))*10**(0.001*eto)  #Find the source

In [32]:
#calculate eff rainfall for each row for each month
#This source: http://www.fao.org/docrep/S2022E/s2022e08.htm was initially used but the updated equaiton of the effective rainfall comes form another source: 

for i in range(1,13):
    df['eff_{}'.format(i)]=0
    
for i in range(1,13):
    df.loc[df['prec{}'.format(i)] < 12.5, 'eff_{}'.format(i)] = df['prec{}'.format(i)]/30
    df.loc[df['prec{}'.format(i)] >= 12.5, 'eff_{}'.format(i)] = eff_rainfall(df['prec{}'.format(i)],df['ETo_{}'.format(i)])/30 

In [36]:

def get_harvest_fraction(i,crop,init,late):
    if i != 12:
        current_date = pd.to_datetime((i+1),format='%m')
    else:
        current_date = pd.to_datetime(1,format='%m')
    start = pd.to_datetime(mode.loc[mode['Mode']==crop,init+'_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
    length = mode.loc[mode['Mode']==crop,init+'_days'].iloc[0]
    days = ((current_date - start).iloc[0].days) % 365
    late_end = pd.to_datetime(mode.loc[mode['Mode']==crop,late+'_end'], format='%d/%m').iloc[0]
    all_days = ((late_end - start).iloc[0].days+1) % 365
    if all_days == 0:
        all_days = 365
   
    if days == 0:
        start = pd.to_datetime(mode.loc[mode['Mode']==crop,late+'_start'], format='%d/%m') #defining the plant start date from excel and setting the correct month and days sequence to read.
        length = mode.loc[mode['Mode']==crop,late+'_days'].iloc[0]
        days = ((current_date - start).iloc[0].days) % 365
        if days <= length:
            return 1 #- days / length
        else:
            return 0
    elif days <= length:
        return days / length
    elif days <= all_days:
        return 1
    else:
        return 0

In [37]:
for i in range (1,13):
    df['PCWR_{}'.format(i)]=0       #PCWR: Peak Crop Water Requirement (l/s/ha) or "Duty", Previously PDWR
    df['PWD_{}'.format(i)]=0        #PWD: Peak Water Demand in (l/s)
    df['SSWD_{}'.format(i)]=0       #SSWD: Seasonal Scheme Water Demand in (m3)
    
#STEP 1: Compute the ACWR from ETc - check FAO1992- page 43-

#acwr=row['ETo_{}'.format(i)]*30*row['kc_{}'.format(i)] - row['eff_{}'.format(i)]*30 - row['awc']/12 ))
#once the available water content layer is obtained, the last past should be added to the equation

for crop in mode['Mode']:
    for i in range(1,13):
        eto = f'ETo_{i}'
        kc = f'kc_{i}'
        eff = f'eff_{i}'
        acwr = f'ACWR_{i}_'+crop
        pcwr = f'PCWR_{i}'
        pwd = f'PWD_{i}'
        sswd = f'SSWD_{i}'
        df[kc+'_'+crop] = float(mode.loc[mode['Mode']==crop,kc])
        ky=ky_dict[crop] #Yield response factor coeff = 0.8 for date palms, source TABLE 53-FAO: http://www.fao.org/3/y4360e/y4360e0b.htm 
        df[acwr] = (df[eto]*30*df[kc+'_'+crop]*ky - df[eff]*30 - (0.12*df[eff])*30) #Assumption: awc=12% effective rainfall
        df.loc[df[acwr]<0,acwr] = 0
        df[pcwr] += ((df[acwr]*10)/30)*2*0.012
        
        df[f'harvest_{i}_'+crop] = df['area_ha'] * np.array([x[crop] for x in df['Mode']]) * get_harvest_fraction(i,crop,'init','late')
        df[pwd] += (df[pcwr] *(df[f'harvest_{i}_'+crop]*24))/(pumping_hours_per_day*aeff*deff)
        df[sswd] += (df[acwr]*10*(df[f'harvest_{i}_'+crop])/(aeff*deff))

In [38]:
# Calculating the annual precipitation: which is the sum of precipitation values

df['precipitation_annual']=df.filter(like='prec').sum(axis=1)  #Filter is used to specify the column of interest


In [39]:
final=df.groupby('NAME_1').sum()
summary = pd.DataFrame({'Irrigated area (ha)':final['area_ha'],
                        'Water intensity (m3/ha)':final.filter(like='SSWD_').sum(axis=1)/final['area_ha'],
                        'Water demand (Mm3)':final.filter(like='SSWD_').sum(axis=1)/1000000})
summary.round(decimals=3)
summary.T

NAME_1,Adrar,Biskra,Djelfa,El Oued,Gabes,Ghadamis,Ghardaia,Gharyan,Illizi,Jufrah,Kebili,Khenchela,Laghouat,Musrata,Ouargla,Tamanrasset,Tataouine,Tebessa,Tozeur
Irrigated area (ha),24101.816848,5749.995666,3449.972096,68999.985344,2874.983928,5353.213079,34499.998422,8557.616421,1770.396215,7963.713904,27412.511949,1149.970213,5749.994721,21082.919486,32699.346634,4868.857546,1833.775565,4599.970966,9617.446033
Water intensity (m3/ha),15793.791148,10033.720376,10303.348469,11470.236142,10074.150089,10515.471347,12775.158413,10818.727280,14169.601170,15113.240202,11207.277369,10830.580811,10084.528374,8890.634530,12379.121096,15656.343103,10215.633880,10927.332730,10794.142744
Water demand (Mm3),380.659062,57.693849,35.546265,791.446126,28.963020,56.291559,440.742945,92.582518,25.085808,120.357521,307.219625,12.454845,57.985985,187.440532,404.789172,76.228504,18.733180,50.265413,103.812085


In [40]:
df.to_csv(output_file+'.csv',index=False)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(output_file + '.xlsx', engine='xlsxwriter')

## Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='3crops')
summary.to_excel(writer, sheet_name='summary')

## Close the Pandas Excel writer and output the Excel file.
writer.save()